Step 1: Import the Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import csv
from io import StringIO
import pandas as pd
import time
import random
from fake_useragent import UserAgent
import numpy as np
import xlsxwriter

Step 2: Get target URL format

In [4]:
url = "https://www.pro-football-reference.com/years/{0}/{1}.htm"
passing_url = "https://www.pro-football-reference.com/years/2023/passing.htm"
rushing_url = "https://www.pro-football-reference.com/years/2023/rushing.htm"
receiving_url = "https://www.pro-football-reference.com/years/2023/receiving.htm"
defense_url = "https://www.pro-football-reference.com/years/2023/defense.htm"
kicking_url = "https://www.pro-football-reference.com/years/2023/kicking.htm"
punting_url = "https://www.pro-football-reference.com/years/2023/punting.htm"
returns_url = "https://www.pro-football-reference.com/years/2023/returns.htm"


<!-- Step 3: Create player data webscraping code -->

In [5]:
def year_parser(site,year,stat):
    year = str(year)
    response = requests.get(site.format(year,stat))
    stat = stat
    if response.status_code == 429:
        print("Too many requests")
    elif response.status_code != 200 and response.status_code != 429:
        print("Error connecting: " + str(response.status_code))
    else:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Locate the table by its ID
        table = soup.find('table', id=stat)

        # Extract headers
        if stat == "passing" or stat == "receiving":
            headers = [th.getText() for th in table.find_all('tr')[0].find_all('th')]
                # Extract rows
            rows = table.find_all('tr')[1:]  # Skip header row

            # Extract data
            data = []
            for row in rows:
                cols = row.find_all('td')
                data.append([col.getText() for col in cols])

            # Create a DataFrame
            df = pd.DataFrame(data, columns=headers[1:])  # Skip the first header as it's for the row labels
        else:
            headers = [th.getText() for th in table.find_all('tr')[1].find_all('th')]
            # Extract rows
            rows = table.find_all('tr')[2:]  # Skip header row

            # Extract data
            data = []
            for row in rows:
                cols = row.find_all('td')
                data.append([col.getText().replace('*','').replace('+','') for col in cols])

            # Create a DataFrame
            df = pd.DataFrame(data, columns=headers[1:])  # Skip the first header as it's for the row labels
        return df

In [6]:
def fetch_w_delay(url, year, stat):
    retry_delay = 1
    time.sleep(retry_delay)
    retry_delay += random.uniform(-2,2)
    df = year_parser(url,year,stat)
    return df

In [9]:
years = [2017, 2018, 2019, 2020, 2021, 2022, 2023,2024]

def grabber(allstats):
    allstats = allstats
    df_dict = {}
    data_dict  = {}
    count = 1
    for x in range(len(years)):
        for i in range(len(allstats)):
            names = str(years[x])+allstats[i]
            targe = (str(years[x]),allstats[i])
            df_dict[names] =  year_parser(url, *targe)
            print(str(count)+". parsed "+names)
            count += 1
            time.sleep(10)
    for key, value in df_dict.items():
        if value is None:
            print("Could not update "+key+" or it would be overwritten")
        else:
            data_dict[key] = value
            print("Successful update of "+key)

    return data_dict

<!-- Step 4: Run Webscrape -->

In [ ]:
targets = ["rushing", "passing", "defense", "receiving", "kicking", "punting", "returns"]
#["rushing", "passing", "defense", "receiving", "kicking", "punting", "returns"]
data_dict = grabber(targets)

In [78]:
player_data = data_dict

In [134]:
team_list = player_data["2024kicking"].Tm.unique()

In [ ]:
team_list

In [153]:
for x in range(len(team_list)):
    team_list[x] = team_list[x].lower()

In [156]:
replacements = {"bal":"rav", "hou":"htx", "lvr":"rai", "ari":"crd", "lac":"sdg", "lar":"ram", "ten":"oti", "ind":"clt"}

In [157]:
updated_teamlist = [replacements.get(s,s) for s in team_list]

In [161]:
team_list = updated_teamlist

In [77]:
for key, value in player_data.items():
    data_dict[key] == data_dict[key].mask(data_dict[key].eq('None')).dropna()
        

In [89]:
direct = "C:/Users/Saul/OneDrive/Documents/Saul's Folder/Coding/Betting Project/Data"
with pd.ExcelWriter(direct + "/Historical Player Data.xlsx",date_format="YYYY/MM/DD") as writer:
    for key, value in player_data.items():
        player_data[key].to_excel(writer, sheet_name=str(key))

In [ ]:
data_dict["2024passing"]

In [ ]:
df

In [ ]:
for x in df:
    if x is None:
        print(True)
    else:
        print(False)

<!-- Step 5: Create Team Roster Webscraping Code -->

In [114]:
def scrape_rosters(team_abbr,year):
    # URL of the roster page for the given team and year
    url = f"https://www.pro-football-reference.com/teams/{team_abbr}/{year}_roster.htm"
    
    # Fetch the page
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch the page. Status code: {response.status_code}")
        return None

    # Parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Function to extract and clean data from the table
    def extract_table_data(table):
        if not table:
            return None
        headers = [th.getText() for th in table.find_all('tr')[0].find_all('th')]
        rows = table.find_all('tr')[1:]  # Skip header row
        table_data = []
        for row in rows:
            cols = row.find_all('td')
            if cols:
                # Clean player names by removing * and † symbols
                player_name = cols[0].getText().replace('*', '').replace('†', '').strip()
                
                # Extract other columns (position, games played, etc.)
                row_data = [player_name] + [col.getText() for col in cols[1:]]
                
                # Append the row data to the list
                table_data.append(row_data)
        return pd.DataFrame(table_data, columns=headers[1:])  # Exclude first header (row label)

    # Function to find the "roster" table (either in regular HTML or in comments)
    def find_table(soup, table_id):
        # First try finding the table directly in the HTML
        table = soup.find('table', id=table_id)
        if table:
            return table

        # If not found, try searching inside HTML comments
        comments = soup.find_all(string=lambda text: isinstance(text, str) and '<table' in text)
        for comment in comments:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            table = comment_soup.find('table', id=table_id)
            if table:
                return table

        return None

    # Find the "roster" table (either in regular HTML or in comments)
    roster_table = find_table(soup, 'roster')

    # Extract data from the roster table
    if roster_table:
        df_roster = extract_table_data(roster_table)
        # Insert the year column
        df_roster.insert(0, 'Year', year)
        return df_roster
    else:
        print(f"No roster table found for {team_abbr} in {year}.")
        return None



In [144]:
def rostgrabber(teams):
    df_dict = {}
    rost_dict  = {}
    count = 1
    for x in range(len(years)):
        for i in range(len(teams)):
            names = str(years[x])+teams[i]
            targe = (teams[i],str(years[x]))
            df_dict[names] =  scrape_rosters(*targe)
            print(str(count)+". parsed "+names)
            count += 1
            time.sleep(10)
    for key, value in df_dict.items():
        if value is None:
            print("Could not update "+key+" or it would be overwritten")
            continue
        else:
            rost_dict[key] = value
            print("Successful update of "+key)

    return rost_dict

In [ ]:
roster_data = rostgrabber(team_list)

In [ ]:
roster_data["2024clt"]

In [165]:
for key, value in roster_data.items():
    roster_data[key] = roster_data[key].drop(roster_data[key].index[-1])

In [170]:
with pd.ExcelWriter(direct + "/Historical Rosters.xlsx",date_format="YYYY/MM/DD") as writer:
    for key, value in roster_data.items():
        roster_data[key].to_excel(writer, sheet_name=str(key))

Load in Data (if needed)

### Step 5: Preprocess the Data

In [189]:
def preprocess_data(df):
    # Handle missing values
    df = df.dropna()

    # Convert categorical columns to numerical using one-hot encoding
    df = pd.get_dummies(df, columns=['Team', 'Pos'])

    # Normalize numerical columns
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numerical_cols] = (df[numerical_cols] - df[numerical_cols].mean()) / df[numerical_cols].std()

    return df

sports_data_processed = preprocess_data(df)


### Step 6: Define and Train the Neural Network

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

def build_and_train_model(df):
    # Split data into features and target
    X = df.drop('spread', axis=1)  # 'spread' is the target variable
    y = df['spread']

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the neural network model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

    # Evaluate the model
    loss = model.evaluate(X_test, y_test)
    print(f'Test Loss: {loss}')

    return model

model = build_and_train_model(sports_data_processed)


### Step 7: Make Predictions

In [ ]:
def predict_spread(model, new_data):
    # Preprocess new data similarly to training data
    new_data_processed = preprocess_data(new_data)
    
    # Make predictions
    predictions = model.predict(new_data_processed)
    return predictions

# Example prediction
new_data = pd.DataFrame([{'team_home': 'Team A', 'team_away': 'Team B', ...}])  # Fill with appropriate features
spread_prediction = predict_spread(model, new_data)
print(spread_prediction)


### Notes
- Adjust the data preprocessing steps based on the specific structure and requirements of your data.
- Ensure the target variable (spread in this example) is correctly defined and available in your dataset.
